# Collaboration and Competition

---

### Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
import sys
sys.path.insert(0, "python/")

from unityagents import UnityEnvironment
import numpy as np
import torch

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Soccer.app"`
- **Windows** (x86): `"path/to/Soccer_Windows_x86/Soccer.exe"`
- **Windows** (x86_64): `"path/to/Soccer_Windows_x86_64/Soccer.exe"`
- **Linux** (x86): `"path/to/Soccer_Linux/Soccer.x86"`
- **Linux** (x86_64): `"path/to/Soccer_Linux/Soccer.x86_64"`
- **Linux** (x86, headless): `"path/to/Soccer_Linux_NoVis/Soccer.x86"`
- **Linux** (x86_64, headless): `"path/to/Soccer_Linux_NoVis/Soccer.x86_64"`

For instance, if you are using a Mac, then you downloaded `Soccer.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Soccer.app")
```

In [2]:
env = UnityEnvironment(file_name="Soccer_Env/Soccer.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 2
        Number of External Brains : 2
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: GoalieBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 112
        Number of stacked Vector Observation: 3
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 
Unity brain name: StrikerBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 112
        Number of stacked Vector Observation: 3
        Vector Action space type: discrete
        Vector Action space size (per agent): 6
        Vector Action descriptions: , , , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we obtain separate brains for the striker and goalie agents.

In [3]:
# print the brain names
print(env.brain_names)

# set the goalie brain
g_brain_name = env.brain_names[0]
g_brain = env.brains[g_brain_name]

# set the striker brain
s_brain_name = env.brain_names[1]
s_brain = env.brains[s_brain_name]

['GoalieBrain', 'StrikerBrain']


### Obtain the State and Action Spaces

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)

# number of agents 
num_g_agents = len(env_info[g_brain_name].agents)
print('Number of goalie agents:', num_g_agents)
num_s_agents = len(env_info[s_brain_name].agents)
print('Number of striker agents:', num_s_agents)

# number of actions
g_action_size = g_brain.vector_action_space_size
print('Number of goalie actions:', g_action_size)
s_action_size = s_brain.vector_action_space_size
print('Number of striker actions:', s_action_size)

# examine the state space 
g_states = env_info[g_brain_name].vector_observations
g_state_size = g_states.shape[1]
print('There are {} goalie agents. Each receives a state with length: {}'.format(g_states.shape[0], g_state_size))
s_states = env_info[s_brain_name].vector_observations
s_state_size = s_states.shape[1]
print('There are {} striker agents. Each receives a state with length: {}'.format(s_states.shape[0], s_state_size))

Number of goalie agents: 2
Number of striker agents: 2
Number of goalie actions: 4
Number of striker actions: 6
There are 2 goalie agents. Each receives a state with length: 336
There are 2 striker agents. Each receives a state with length: 336


### DQN Agent

Attempt to implement DQN, leveraging Udacity Deep Reinforcement Learning [repo](https://github.com/udacity/deep-reinforcement-learning)

In [5]:
from dqn.dqn_agent_v2 import Agent
from collections import deque

g_agent = Agent(state_size=g_state_size, action_size=g_action_size, seed=0)
s_agent = Agent(state_size=s_state_size, action_size=s_action_size, seed=0)

Attempt to modify reward structure to incentivize going after the ball

In [6]:
def ball_reward(state):
    """
    Params
    ======
        state : current state of striker, 3 stacked 112 element vector
            1: ball
            2: opponent's goal
            3: own goal
            4: wall
            5: teammate
            6: opponent
            7: distance
    """
    reward = 0.0
    # Penalize if ball is not in view
    if not any(state[0::8]):
        reward = -0.03
    # Reward for kicking the ball
    else:
        idx = np.where(state[0::8])[0] # check which ray sees the ball
        distance = state[idx*8 + 7] # get the corresponding distance to ball
        if (np.amin(distance) <= 0.03): # Just picking some thresholds for now.
            reward = 0.3

    return reward

In [12]:
def dqn(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    g_losses = []
    g_losses_window = deque(maxlen=100)
    s_losses = []
    s_losses_window = deque(maxlen=100)
    for i_episode in range(1, n_episodes+1):
        env_info  = env.reset(train_mode=True)
        score = 0
        ball_reward_val = 0.0
        
        g_states = env_info[g_brain_name].vector_observations       # get initial state (goalies)
        s_states = env_info[s_brain_name].vector_observations       # get initial state (strikers)

        g_scores = np.zeros(num_g_agents)                           # initialize the score (goalies)
        s_scores = np.zeros(num_s_agents)                           # initialize the score (strikers)  
        
        for t in range(max_t):
            action_g_0 = g_agent.act(g_states[0], eps)        # always pick state index 0
            action_s_0 = s_agent.act(s_states[0], eps)  
            
            # Set other team to random
            action_g_1 = np.asarray( [np.random.choice(g_action_size)] ) 
            action_s_1 = np.asarray( [np.random.choice(s_action_size)] )
            # Train simultaneously
            #action_g_1 = g_agent.act(g_states[1], eps)        # always pick state index 1
            #action_s_1 = s_agent.act(s_states[1], eps) 
            
            # Combine actions
            actions_g = np.array( (action_g_0, action_g_1) )                                    
            actions_s = np.array( (action_s_0, action_s_1) )
            actions = dict( zip( [g_brain_name, s_brain_name], [actions_g, actions_s] ) )
            
            env_info = env.step(actions)                                                
            # get next states
            g_next_states = env_info[g_brain_name].vector_observations         
            s_next_states = env_info[s_brain_name].vector_observations
            
            # get reward and update scores
            g_rewards = env_info[g_brain_name].rewards
            s_rewards = env_info[s_brain_name].rewards
            g_scores += g_rewards
            s_scores += s_rewards
            
            ball_reward_val += ball_reward(s_states[0])
            
            # check if episode finished
            done = np.any(env_info[g_brain_name].local_done)
            
            # store experiences
            g_agent.step(g_states[0], action_g_0, g_rewards[0], 
                         g_next_states[0], done)
            s_agent.step(s_states[0], action_s_0, s_rewards[0] + ball_reward(s_states[0]), # adding ball reward
                         s_next_states[0], done)

            if done:
                break
                
            g_states = g_next_states
            s_states = s_next_states
                
        # learn
        goalie_loss = g_agent.learn(g_agent.memory.sample(), 0.99) # discount = 0.99
        striker_loss = s_agent.learn(s_agent.memory.sample(), 0.99) # discount = 0.99 
        
        g_losses.append(goalie_loss.item())
        g_losses_window.append(goalie_loss.item())
        #print(goalie_loss.item())
        s_losses.append(striker_loss.item())
        s_losses_window.append(striker_loss.item())
        
        score = g_scores[0] + s_scores[0]
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}\t Goalie Loss:' \
                  '{:.5f}\t Striker Loss: {:.5f}' \
                  '\t Ball Reward: {:.2f}'.format(i_episode, \
                                                  np.mean(scores_window), \
                                                  np.mean(g_losses_window), \
                                                  np.mean(s_losses_window), \
                                                  ball_reward_val), end="")
        #print(s_states[0][0:56])
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}\t Goalie Loss:' \
                  '{:.5f}\t Striker Loss: {:.5f}\n' \
                  '\t Ball Reward: {:.2f}'.format(i_episode, \
                                                  np.mean(scores_window), \
                                                  np.mean(g_losses_window), \
                                                  np.mean(s_losses_window), \
                                                  ball_reward_val))
            torch.save(g_agent.qnetwork_local.state_dict(), 'checkpoint_goalie.pth')
            torch.save(s_agent.qnetwork_local.state_dict(), 'checkpoint_striker.pth')
    return scores


Set hyperparameters and train DQN

In [13]:
n_episodes = 5000
#n_episodes = 1
max_t = 100000
eps_start = 1.0
eps_end = 0.1
eps_decay = 0.9995

# Pick up where we left off
#GOALIE = './trained_models/goalie_dqn_run1.pth'
#STRIKER = './trained_models/striker_dqn_run1.pth'
#g_agent.qnetwork_local.load (GOALIE )
#s_agent.qnetwork_local.load( STRIKER )
GOALIE = './trained_models/goalie_dqn_ballreward2.pth'
STRIKER = './trained_models/striker_dqn_ballreward2.pth'
g_agent.qnetwork_local.load (GOALIE )
s_agent.qnetwork_local.load( STRIKER )

# Train
eps_start = 0.1
eps_end = 0.1
scores = dqn(n_episodes, max_t, eps_start, eps_end, eps_decay)

Episode 100	Average Score: 0.35	 Goalie Loss:0.00071	 Striker Loss: 0.01799	 Ball Reward: -3.42
	 Ball Reward: -3.42
Episode 200	Average Score: 0.04	 Goalie Loss:0.00117	 Striker Loss: 0.00294	 Ball Reward: -14.31
	 Ball Reward: -14.31
Episode 300	Average Score: -0.22	 Goalie Loss:0.00103	 Striker Loss: 0.00334	 Ball Reward: 31.507
	 Ball Reward: 31.50
Episode 400	Average Score: -0.24	 Goalie Loss:0.00168	 Striker Loss: 0.00223	 Ball Reward: -0.723
	 Ball Reward: -0.72
Episode 500	Average Score: -0.03	 Goalie Loss:0.00101	 Striker Loss: 0.00259	 Ball Reward: -4.771
	 Ball Reward: -4.77
Episode 600	Average Score: 0.07	 Goalie Loss:0.00115	 Striker Loss: 0.00190	 Ball Reward: 3.5187
	 Ball Reward: 3.51
Episode 700	Average Score: 0.02	 Goalie Loss:0.00101	 Striker Loss: 0.00266	 Ball Reward: -16.988
	 Ball Reward: -16.98
Episode 800	Average Score: 0.04	 Goalie Loss:0.00066	 Striker Loss: 0.00184	 Ball Reward: 0.4826
	 Ball Reward: 0.48
Episode 900	Average Score: -0.04	 Goalie Loss:0.00131

In [ ]:
# Load trained agents and run

# GOALIE = './trained_models/goalie_dqn_run1.pth'
# STRIKER = './trained_models/striker_dqn_run1.pth'
GOALIE = './trained_models/goalie_dqn_ballreward2.pth'
STRIKER = './trained_models/striker_dqn_ballreward2.pth'
g_agent.qnetwork_local.load (GOALIE )
s_agent.qnetwork_local.load( STRIKER )

### Test Trained Agent

In [ ]:
for i in range(500):                                         # play game for 2 episodes
    env_info = env.reset(train_mode=False)                 # reset the environment    
    g_states = env_info[g_brain_name].vector_observations  # get initial state (goalies)
    s_states = env_info[s_brain_name].vector_observations  # get initial state (strikers)
    g_scores = np.zeros(num_g_agents)                      # initialize the score (goalies)
    s_scores = np.zeros(num_s_agents)                      # initialize the score (strikers)
    while True:
        # select actions and send to environment
        action_g_0 = g_agent.act(g_states[0], 0)        # always pick state index 0
        action_s_0 = s_agent.act(s_states[0], 0)  

        # Set other team to random
        action_g_1 = np.asarray( [np.random.choice(g_action_size)] ) 
        action_s_1 = np.asarray( [np.random.choice(s_action_size)] )

        # Combine actions
        actions_g = np.array( (action_g_0, action_g_1) )                                    
        actions_s = np.array( (action_s_0, action_s_1) )
        actions = dict( zip( [g_brain_name, s_brain_name], [actions_g, actions_s] ) )

        env_info = env.step(actions)                       
        
        # get next states
        g_next_states = env_info[g_brain_name].vector_observations         
        s_next_states = env_info[s_brain_name].vector_observations
        
        # get reward and update scores
        g_rewards = env_info[g_brain_name].rewards  
        s_rewards = env_info[s_brain_name].rewards
        g_scores += g_rewards
        s_scores += s_rewards
        
        # check if episode finished
        done = np.any(env_info[g_brain_name].local_done)  
        
        # roll over states to next time step
        g_states = g_next_states
        s_states = s_next_states
        
        # exit loop if episode finished
        if done:                                           
            break
    print('Scores from episode {}: {} (goalies), {} (strikers)'.format(i+1, g_scores, s_scores))

### Test - Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [ ]:
for i in range(2):                                         # play game for 2 episodes
    env_info = env.reset(train_mode=False)                 # reset the environment    
    g_states = env_info[g_brain_name].vector_observations  # get initial state (goalies)
    s_states = env_info[s_brain_name].vector_observations  # get initial state (strikers)
    g_scores = np.zeros(num_g_agents)                      # initialize the score (goalies)
    s_scores = np.zeros(num_s_agents)                      # initialize the score (strikers)
    while True:
        # select actions and send to environment
        g_actions = np.random.randint(g_action_size, size=num_g_agents)
        s_actions = np.random.randint(s_action_size, size=num_s_agents)
        actions = dict(zip([g_brain_name, s_brain_name], 
                           [g_actions, s_actions]))
        env_info = env.step(actions)                       
        
        # get next states
        g_next_states = env_info[g_brain_name].vector_observations         
        s_next_states = env_info[s_brain_name].vector_observations
        
        # get reward and update scores
        g_rewards = env_info[g_brain_name].rewards  
        s_rewards = env_info[s_brain_name].rewards
        g_scores += g_rewards
        s_scores += s_rewards
        
        # check if episode finished
        done = np.any(env_info[g_brain_name].local_done)  
        
        # roll over states to next time step
        g_states = g_next_states
        s_states = s_next_states
        
        # exit loop if episode finished
        if done:                                           
            break
    print('Scores from episode {}: {} (goalies), {} (strikers)'.format(i+1, g_scores, s_scores))

In [14]:
env.close()